In [0]:
dbutils.fs.ls("FileStore/tables/")

Out[7]: [FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2019_csv.gz', name='clinicaltrial_2019_csv.gz', size=10060669, modificationTime=1650757928000),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2020_csv.gz', name='clinicaltrial_2020_csv.gz', size=10981608, modificationTime=1650757932000),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2021.csv', name='clinicaltrial_2021.csv', size=50359696, modificationTime=1652388947000),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2021_csv.gz', name='clinicaltrial_2021_csv.gz', size=11921810, modificationTime=1650757976000),
 FileInfo(path='dbfs:/FileStore/tables/mesh-1.csv', name='mesh-1.csv', size=5295548, modificationTime=1652223661000),
 FileInfo(path='dbfs:/FileStore/tables/mesh.csv', name='mesh.csv', size=5295548, modificationTime=1651020967000),
 FileInfo(path='dbfs:/FileStore/tables/pharma-1.csv', name='pharma-1.csv', size=619715, modificationTime=1652223644000),
 FileInfo(path='dbfs:/FileStore/tables/pharma.csv

In [0]:
dbutils.fs.cp("FileStore/tables/clinicaltrial_2021_csv.gz","file:/tmp/")

Out[8]: True

In [0]:
%sh
gzip -d /tmp/clinicaltrial_2021_csv.gz

In [0]:
%sh
ls /tmp

Rserv
Rtmp1FxNq4
chauffeur-daemon-params
chauffeur-daemon.pid
chauffeur-env.sh
clinicaltrial_2021_csv
custom-spark.conf
driver-daemon-params
driver-daemon.pid
driver-env.sh
hsperfdata_root
ipykernel-connection-ReplId-176df-0bcec-d4ba9-8.json
ipykernel-connection-ReplId-35414-b6389-cfdd5-6.json
ipykernel-connection-ReplId-3d793-35f85-795ee-9.json
ipykernel-connection-ReplId-3e0c7-444ab-2638d-c.json
systemd-private-b2377b96cd7d4635bc7ed475b33154f7-apache2.service-iqBeUh
systemd-private-b2377b96cd7d4635bc7ed475b33154f7-ntp.service-3QIaWe
systemd-private-b2377b96cd7d4635bc7ed475b33154f7-systemd-logind.service-5Othvg
systemd-private-b2377b96cd7d4635bc7ed475b33154f7-systemd-resolved.service-tUGo2f
tmp.3EaIBO8h3n


In [0]:
mv /tmp/clinicaltrial_2021_csv /tmp/clinicaltrial_2021.csv

In [0]:
dbutils.fs.mv("file:/tmp/clinicaltrial_2021.csv","/FileStore/tables/",True)

Out[12]: True

In [0]:
dbutils.fs.ls("/FileStore/tables/")

Out[13]: [FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2019_csv.gz', name='clinicaltrial_2019_csv.gz', size=10060669, modificationTime=1650757928000),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2020_csv.gz', name='clinicaltrial_2020_csv.gz', size=10981608, modificationTime=1650757932000),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2021.csv', name='clinicaltrial_2021.csv', size=50359696, modificationTime=1652388968000),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2021_csv.gz', name='clinicaltrial_2021_csv.gz', size=11921810, modificationTime=1650757976000),
 FileInfo(path='dbfs:/FileStore/tables/mesh-1.csv', name='mesh-1.csv', size=5295548, modificationTime=1652223661000),
 FileInfo(path='dbfs:/FileStore/tables/mesh.csv', name='mesh.csv', size=5295548, modificationTime=1651020967000),
 FileInfo(path='dbfs:/FileStore/tables/pharma-1.csv', name='pharma-1.csv', size=619715, modificationTime=1652223644000),
 FileInfo(path='dbfs:/FileStore/tables/pharma.cs

In [0]:
clinicaltrial_2021=spark.read.csv("/FileStore/tables/clinicaltrial_2021.csv",sep="|",header=True)
clinicaltrial_2021.take(10)

Out[14]: [Row(Id='NCT02758028', Sponsor='The University of Hong Kong', Status='Recruiting', Start='Aug 2005', Completion='Nov 2021', Type='Interventional', Submission='Apr 2016', Conditions=None, Interventions=None),
 Row(Id='NCT02751957', Sponsor='Duke University', Status='Completed', Start='Jul 2016', Completion='Jul 2020', Type='Interventional', Submission='Apr 2016', Conditions='Autistic Disorder,Autism Spectrum Disorder', Interventions=None),
 Row(Id='NCT02758483', Sponsor='Universidade Federal do Rio de Janeiro', Status='Completed', Start='Mar 2017', Completion='Jan 2018', Type='Interventional', Submission='Apr 2016', Conditions='Diabetes Mellitus', Interventions=None),
 Row(Id='NCT02759848', Sponsor='Istanbul Medeniyet University', Status='Completed', Start='Jan 2012', Completion='Dec 2014', Type='Observational', Submission='May 2016', Conditions='Tuberculosis,Lung Diseases,Pulmonary Disease', Interventions=None),
 Row(Id='NCT02758860', Sponsor='University of Roma La Sapienza', 

In [0]:
#Problem1
clinicaltrial_2021.count()

Out[15]: 387261

In [0]:
#Problem2
from pyspark.sql.functions import desc
#Frequency of Type of treatment
cltrial_count_df=clinicaltrial_2021.groupBy('Type').count().sort(desc("count"))
cltrial_count_df.show()

+--------------------+------+
|                Type| count|
+--------------------+------+
|      Interventional|301472|
|       Observational| 77540|
|Observational [Pa...|  8180|
|     Expanded Access|    69|
+--------------------+------+



In [0]:
#Problem3
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.functions import desc
clinicaltrial_2021a=clinicaltrial_2021.select("Conditions",explode(split("Conditions",","))).dropna()
#Frequency of top 5 Conditions
cltrial_countcond_df=clinicaltrial_2021a.groupBy("col").count().sort(desc("count"))
cltrial_countcond_df.show(5)

+-----------------+-----+
|              col|count|
+-----------------+-----+
|        Carcinoma|13389|
|Diabetes Mellitus|11080|
|        Neoplasms| 9371|
| Breast Neoplasms| 8640|
|         Syndrome| 8032|
+-----------------+-----+
only showing top 5 rows



In [0]:
#Problem4
from pyspark.sql.functions import col
mesh=spark.read.csv("/FileStore/tables/mesh.csv",sep=',',header=True)
mesh.show()
cltrial_mesh_df=clinicaltrial_2021.select(['Id','Conditions']).withColumn('Conditions',explode(split('Conditions',',')))
cltrial_mesh_df=mesh.join(cltrial_mesh_df,cltrial_mesh_df.Conditions==mesh.term,how='inner')
cltrial_mesh_df.show(10)
cltrial_mesh_df1=cltrial_mesh_df.withColumn('Tree', col('tree').substr(1,3)).groupBy('Tree').count()
#Intervention frequency top 10 dataframe
cltrial_meshsol_df=cltrial_mesh_df1.orderBy('count',ascending=False)
cltrial_meshsol_df.show(10)

+--------------------+--------------------+
|                term|                tree|
+--------------------+--------------------+
|          Calcimycin| D03.633.100.221.173|
|             A-23187| D03.633.100.221.173|
|             Temefos| D02.705.400.625.800|
|             Temefos| D02.705.539.345.800|
|             Temefos| D02.886.300.692.800|
|               Abate| D02.705.400.625.800|
|               Abate| D02.705.539.345.800|
|               Abate| D02.886.300.692.800|
|               Difos| D02.705.400.625.800|
|               Difos| D02.705.539.345.800|
|               Difos| D02.886.300.692.800|
|           Abattoirs|J01.576.423.200.7...|
|           Abattoirs|         J03.540.020|
|Abbreviations as ...|L01.559.598.400.5...|
|   Acronyms as Topic|L01.559.598.400.5...|
|             Abdomen|         A01.923.047|
|      Abdomen, Acute|C23.888.592.612.0...|
|      Abdomen, Acute| C23.888.821.030.249|
|  Abdominal Injuries|             C26.017|
| Abdominal Neoplasms|         C

In [0]:
#Problem5
pharma=spark.read.csv("/FileStore/tables/pharma.csv",sep=',',header=True)
pharma.show()
cltrial_pharma_df1=pharma.select(['Parent_Company'])
cltrial_pharma_df1.show()
cltrial_pharma_df2=clinicaltrial_2021.select(['Sponsor'])
cltrial_pharma_df2.show()
from pyspark.sql.functions import col

cltrial_pharm_df3=cltrial_pharma_df2.join(cltrial_pharma_df1,cltrial_pharma_df1.Parent_Company==cltrial_pharma_df2.Sponsor,'leftanti')
cltrial_pharm_df3.show()
#Frequency of Sponsors without Private Companies
cltrial_pharma_df4=cltrial_pharm_df3.groupBy('Sponsor').count().orderBy('count',ascending=False)
cltrial_pharma_df4.show(10)

+--------------------+--------------------+--------------+------------------------+---------------------------------------------------------+------------+------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------+--------------------+------------------+---------------------+-----+---------------+-----------------------------+------------------+--------------+------------+--------------------+-----+----------+--------------------+--------------------+------------------+-------------------+---------------------------+------------------------+---------------------------+--------------------+--------------------+
|             Company|      Parent_Company|Penalty_Amount|Subtraction_From_Penalty|Penalty_Amount_Adjusted_For_Eliminating_Multiple_Counting|Penalty_Year|Penalty_Date|       Offense_Group|     Primary_Offense|   Secondary_Offense|         Description|Level_of_Government|  Action_Type|              Agency|    C

In [0]:
#Problem6
from pyspark.sql.functions import col
from pyspark.sql.functions import split
cltrial_date_df=clinicaltrial_2021.where(clinicaltrial_2021.Status=='Completed')
#cltrial_date_df.show()
cltrial_date_df1=cltrial_date_df.withColumn("Completion_month",split(col("Completion")," ").getItem(0)).withColumn("Completion_year",split(col("Completion")," ").getItem(1))
#cltrial_date_df1.show()
cltrial_date_df2=cltrial_date_df1.where(cltrial_date_df1.Completion_year=="2021")
#cltrial_date_df2.show()
cltrial_date_df3=cltrial_date_df2.groupBy('Completion_month').count().orderBy('count')
cltrial_date_df3.show()
cltrial_mon=spark.createDataFrame([("Jan"),("Feb"),("Mar"),("Apr"),("May"),("Jun"),("Jul"),("Aug"),("Sep"),("Oct")],"string").toDF("Month")
cltrial_mon.show()
#Order by Month in ascending order of Completion month frequency
cltrial_month=cltrial_date_df3.join(cltrial_mon,cltrial_date_df3.Completion_month==cltrial_mon.Month,'right')
cltrial_month=cltrial_month.drop(cltrial_month.Completion_month)
cltrial_month.show()

+----------------+-----+
|Completion_month|count|
+----------------+-----+
|             Oct|  187|
|             Sep|  528|
|             Aug|  700|
|             Jul|  819|
|             Feb|  934|
|             Apr|  967|
|             May|  984|
|             Jun| 1094|
|             Jan| 1131|
|             Mar| 1227|
+----------------+-----+

+-----+
|Month|
+-----+
|  Jan|
|  Feb|
|  Mar|
|  Apr|
|  May|
|  Jun|
|  Jul|
|  Aug|
|  Sep|
|  Oct|
+-----+

+-----+-----+
|count|Month|
+-----+-----+
| 1131|  Jan|
|  934|  Feb|
| 1227|  Mar|
|  967|  Apr|
|  984|  May|
| 1094|  Jun|
|  819|  Jul|
|  700|  Aug|
|  528|  Sep|
|  187|  Oct|
+-----+-----+



In [0]:
cltrial_date_Jan=cltrial_date_df2.where(cltrial_date_df2.Completion_month=='Jan').show()
cltrial_date_Feb=cltrial_date_df2.where(cltrial_date_df2.Completion_month=='Feb').show()
cltrial_date_Mar=cltrial_date_df2.where(cltrial_date_df2.Completion_month=='Mar').show()
cltrial_date_April=cltrial_date_df2.where(cltrial_date_df2.Completion_month=='April').show()
cltrial_date_May=cltrial_date_df2.where(cltrial_date_df2.Completion_month=='May').show()
cltrial_date_Jun=cltrial_date_df2.where(cltrial_date_df2.Completion_month=='Jun').show()
cltrial_date_July=cltrial_date_df2.where(cltrial_date_df2.Completion_month=='Jul').show()
cltrial_date_Aug=cltrial_date_df2.where(cltrial_date_df2.Completion_month=='Aug').show()
cltrial_date_Sep=cltrial_date_df2.where(cltrial_date_df2.Completion_month=='Sep').show()
cltrial_date_Oct=cltrial_date_df2.where(cltrial_date_df2.Completion_month=='Oct').show()
cltrial_date_Nov=cltrial_date_df2.where(cltrial_date_df2.Completion_month=='Nov').show()
cltrial_date_Dec=cltrial_date_df2.where(cltrial_date_df2.Completion_month=='Dec').show()

+-----------+--------------------+---------+--------+----------+--------------------+----------+--------------------+--------------------+----------------+---------------+
|         Id|             Sponsor|   Status|   Start|Completion|                Type|Submission|          Conditions|       Interventions|Completion_month|Completion_year|
+-----------+--------------------+---------+--------+----------+--------------------+----------+--------------------+--------------------+----------------+---------------+
|NCT02753530|           Orphazyme|Completed|Aug 2017|  Jan 2021|      Interventional|  Apr 2016|            Myositis|                null|             Jan|           2021|
|NCT02758782|Charite Universit...|Completed|Sep 2016|  Jan 2021|      Interventional|  Apr 2016|         Spondylitis| Celecoxib,Golimumab|             Jan|           2021|
|NCT03095209|Fox Chase Cancer ...|Completed|Feb 2017|  Jan 2021|       Observational|  Mar 2017|Lung Neoplasms,Es...|Paclitaxel,Carbop...|  